In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter

import copy
import math
import json
import os
import random
import numpy as np
import pandas as pd
import time

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import matplotlib.pyplot as plt
%matplotlib inline

import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.utils.data import Dataset, DataLoader
from time import perf_counter 

class LSTMcell(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_size):
        super(LSTMcell, self).__init__()
        self.hidden_size = hidden_dim
        self.output_size = output_size
        # nn.Linear(in_features, out_features, bias=True)
        self.cgate = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.igate = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.fgate = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.ogate = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_size)
        # Activation functions
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self, sample, hidden = None, cellstate = None):
        if hidden is None:
            hidden = torch.zeros(1, self.hidden_size, dtype=sample.dtype, device=sample.device)
        if cellstate is None:
            cellstate = torch.zeros(1, self.hidden_size)

        combined = torch.cat((sample, hidden), 1)

        f_gate = self.sigmoid(self.fgate(combined))
        i_gate = self.sigmoid(self.igate(combined))
        o_gate = self.sigmoid(self.ogate(combined))
        c_tilde = self.tanh(self.cgate(combined))

        first = torch.mul(cellstate, f_gate)
        second = torch.mul(c_tilde, i_gate)
        cell = torch.add(first, second)

        hidden = torch.mul(self.tanh(cell), o_gate)
        output = self.output(hidden)
        # output = self.sigmoid(output)
        return output, hidden, cell
    
class LSTM(nn.Module):
    def __init__(self, p, q, output_size):
        super(LSTM, self).__init__()
        self.p = p
        self.q = q

        self.hidden_dim = q
        self.feature_size = p
        self.output_size = output_size
        self.lstm = LSTMcell(self.feature_size, self.hidden_dim, self.output_size)
 
    def forward(self, input_y, h_0 = None, c_0 = None):      
        time_steps = input_y.shape[0]
        outputs = torch.Tensor(time_steps, self.output_size)
        samples = input_y
        for t in range(time_steps):
            tmp, c, h = self.lstm(samples[t,:].reshape(1,-1), h_0, c_0)
            outputs[t, :] = tmp.view(-1,self.output_size)
        return outputs, c, h

In [2]:
### " F-norm of A
class L2LossFun(nn.Module):
    
    def __init__(self):
        super(L2LossFun, self).__init__()
    def forward(self, A_Est, A_True):
        gap = math.sqrt(np.sum((A_Est - A_True)**2))
        return gap

class LinfLossFun(nn.Module):
    
    def __init__(self):
        super(LinfLossFun, self).__init__()
    def forward(self, A_Est, A_True):
        gap = np.max(np.abs(A_Est-A_True))
        return gap

In [3]:
dfP = pd.read_csv("co2_327_45.csv") 
df = np.array(dfP)
### "Data preprocessing
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)

In [4]:
# Hyper Parameters
N = 45
output_size = 45
Smp_size = 100
q = 1 #hidden dimension
lag = 20
Pred_size = len(df)-Smp_size-lag
# the layer is set to be 1 and fixed

In [5]:
names = {}
names["predErrorLSTM1"] =  []
names["predErrorLSTM1Linf"] =  []
names["LTRLSTM1_pred"] = []

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter
import copy
from copy import deepcopy
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
 
# initialization
p , q, output_size = (N, 1, N)

# transform the time series involving
def create_dataset(y, look_back=1):
    dataYp,dataYc = [], [] #Ypast, Ycurrent
    for i in range(len(y)-look_back):
        dataYp.append(y[i,:])
        dataYc.append(y[i+look_back, :])
    return np.array(dataYp), np.array(dataYc)
 
# transform data to tensor in torch
def to_torch(state):
    state = torch.from_numpy(state).float()
    return state

In [7]:
distance = L2LossFun()
distanceLinf = LinfLossFun()

t1_start = perf_counter() 

for k in range(pred_size):
    y = df[(Smp_size+k+lag-100):(Smp_size+k+lag+1),:] #Fixed size at 100! Need to readjust
    inY, outY = create_dataset(y=y, look_back=1) 
    # split into train and test sets
    train_size = 100                                  #Fixed size at 100! Need to readjust
    test_size = 1
    trainInY,trainOutY = inY[0:train_size-1], outY[0:train_size-1]
    testInY,testOutY =  inY[train_size-1:], outY[train_size-1:]
    trainInY = np.reshape(trainInY, (trainInY.shape[0], p))
    trainOutY = np.reshape(trainOutY, (trainOutY.shape[0], p))
    testInY = np.reshape(testInY, (testInY.shape[0], p))
    testOutY = np.reshape(testOutY, (testOutY.shape[0], p))

    model = LSTM(p = N,q = 1, output_size = output_size)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0

    i = 0
    while abs(loss_last - loss_new)  > 0.000001:
        if i > 0:
            loss_last = loss_new  

        model.zero_grad()
        target = torch.from_numpy(trainOutY).float()
        output,_,_ = model(torch.from_numpy(trainInY).float())

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        output,c,h = model(torch.from_numpy(trainInY).float())
        loss_new = loss.item()
        i = i + 1
        #print(loss_new)
    testPredict,_,_ = model(torch.from_numpy(testInY).float(), h, c)
    testPredict = testPredict.detach().numpy()
    # invert predictions
    #testPredict_r = scaler.inverse_transform(testPredict[:,:])
    #testY_r = scaler.inverse_transform(testOutY[:,:])
    names["LTRLSTM1_pred"].append(testPredict)
    predError = distance(testPredict, testOutY)
    names["predErrorLSTM1"].append(predError)
    print("PredError is {}.".format(predError))
    predErrorLinf = distanceLinf(testPredict, testOutY)
    names["predErrorLSTM1Linf"].append(predErrorLinf)
    print("PredErrorLinf is {}.".format(predErrorLinf))
    
    print(k)

t1_stop = perf_counter() 
print("Elapsed time during the whole program in seconds:", 
                                        t1_stop-t1_start) 
Real_100LSTM2s = names
torch.save(Real_100LSTM2s, "Real_100LSTM2s.py")

PredError is 1.00128117998523.
PredErrorLinf is 0.38144074867613237.
0
PredError is 1.215021955242396.
PredErrorLinf is 0.5131666362285614.
1
PredError is 0.2858962379307188.
PredErrorLinf is 0.09704349387062217.
2
PredError is 0.7905071416117887.
PredErrorLinf is 0.32342204710345657.
3
PredError is 0.6232098228483328.
PredErrorLinf is 0.4286918409996652.
4
PredError is 1.1296482405187542.
PredErrorLinf is 0.41724358076179247.
5
PredError is 0.878437030143141.
PredErrorLinf is 0.32893234491348267.
6
PredError is 0.9050675432532376.
PredErrorLinf is 0.27809623674755346.
7
PredError is 0.6749884263040304.
PredErrorLinf is 0.40698421001434326.
8
PredError is 0.5973564092486257.
PredErrorLinf is 0.27701140486904907.
9
Elapsed time during the whole program in seconds: 482.33623731899934
